#### Parton Shower

In [1]:
from random import * 
import math 
import numpy as np 
import scipy 
from scipy import optimize
from matplotlib import pyplot as plt
from tqdm import tqdm 
from scipy.integrate import quad # for numerical integrals
from alphaS import * 


Inizializzo $\alpha_S$ passando come riferimento la massa dello Z

In [2]:
aS=alphaS(0.118,91.186)

In [3]:
# RUN OPTIONS: parametri fondamentali

# Numero di linee di quark da simulare

Nevolve = 10000

# scala del cutoff
Qc = 1.

# the hard scale, e.g. 1000 GeV
Q = 1000.

In [4]:
def Pqq(z):
    return CF*(1.+z**2)/(1.-z)

def Pqq_over(z):
    return 2*CF/(1.-z)

# calculate the transverse momentum of the emission

def Get_pTsq(t, z): 
    return z**2 * (1-z)**2 * t

# calculate the virtual mass-squared of the emitting particle

def Get_mvirtsq(t,z): 
    return z*(1-z) * t

def alphaS(t, z):

    scale= z * (1-z) * math.sqrt(t)
    
    if scale < Qc:
        return aS.alphasQ(Qc)/2./math.pi
    
    return aS.alphasQ(scale)/2./math.pi


def get_alphaS_over():
    
    scale = Qc # the minimum scale^2 available 
    alphaS_over = aS.alphasQ(scale)/2./math.pi
    
    return alphaS_over

Definiamo anche

In [5]:
def tGamma(z):
    return -2*get_alphaS_over()*CF*np.log(1.-z)

def Inverse_tGamma(y):
     return 1. - np.exp(- 0.5*y/CF/get_alphaS_over())

def zp_over(t):
    return 1-np.sqrt(Qc**2/t)
def zm_over(t):
    return np.sqrt(Qc**2/t)




def Get_zEmission(t,R):
    return Inverse_tGamma(tGamma(zm_over(t))+R*( tGamma(zp_over(t))-tGamma(zm_over(t)) ))

def rho(z1,z2):
    return tGamma(z1)-tGamma(z2)

In [6]:
def EmissionScaleFunc(logt_over_Qsq,Q,R):

    t = Q**2 * np.exp( logt_over_Qsq )
    
    r=rho(zp_over(t),zm_over(t))

    return logt_over_Qsq-(1./r)*np.log(R)


def Get_tEmission(Q, tfac,R):

    tolerance = 1E-3 # the tolerance for the solution

    EmissionFunc_arg = lambda x : EmissionScaleFunc(x, Q,R) # the function in a form appropriate for the solver

    # calculate the solution using "Ridder's" method

    lower_bound=np.log(tfac*Qc**2/Q**2)
    upper_bound=0.

    sol, results = scipy.optimize.ridder(EmissionFunc_arg, lower_bound, upper_bound, xtol=tolerance, full_output=True, maxiter=1000)

    tEm = Q**2 * np.exp( sol )

    # if a solution has not been found, terminate the evolution 
    #        
    if abs(EmissionFunc_arg(sol)) > tolerance:
            return Q**2, [], False
            
    # otherwise return the emission scale and continue
    return tEm, results, True

In [7]:
#
def Generate_Emission(Q, tfac):
    
    generated = True

    R1 = random()
    R2 = random()
    R3 = random() #veto test over z
    R4 = random() #veto test over t
    
    # solve for the (candidate) emission scale:

    tEm, results, continueEvolution = Get_tEmission(Q,tfac,R1)
    
    # if no solution is found then end branch

    if continueEvolution == False:
        zEm = 1.
        pTsqEm = 0.
        MsqEm = 0.
        return tEm, zEm, pTsqEm, MsqEm, generated, continueEvolution

    
    
    # get the (candidate) z of the emission
    zEm = Get_zEmission(tEm,R2)
    
    # get the transverse momentum 
    pTsqEm = Get_pTsq(tEm, zEm)
    # get the virtual mass squared:
    MsqEm = Get_mvirtsq(tEm, zEm)

    # now check the conditions to accept or reject the emission:
    # check if the transverse momentum is physical:
    if pTsqEm < 0.:
      
        generated = False
        
    # compare the splitting function overestimate prob to a random number
    if (Pqq(zEm)/Pqq_over(zEm))*(alphaS(tEm, zEm)/get_alphaS_over()) < R3:
        generated = False
    
    # compare the alphaS overestimate prob to a random number

    #if alphaS(tEm, zEm)/get_alphaS_over() < R4:
    #    generated = False
    
   
    if generated == False: # rejected emission
        zEm = 1.
        pTsqEm = 0.
        MsqEm = 0.
        
    # return all the variables for the emission
    return tEm, zEm, pTsqEm, MsqEm, generated, continueEvolution

In [8]:
# the function that performs the evolution of a single branch:
# returns a list of all the emissions for further processing

def Evolve(Q):
    # the minimum evolution scale
    tEm_min = Qc**2
    # counter for the number of emissions:
    Nem = 0
    # array to store emission info:
    Emissions = []

    fac_tEm = 3.999 # minimum value for the cutoff to try emissions = fac_tEm * Qc**2 (should be less than the actual cutoff)
    fac_cutoff = 4. # actual cutoff = fac_cutoff * Qc**2

    # start the evolution

    tEm = Q**2 # initial value of the evolution variable
    zEm = 1    # initial value of the momentum fraction

    

    while np.sqrt(tEm)*zEm > np.sqrt(fac_cutoff*tEm_min):

        # evolve:
        tEm, zEm, pTsqEm, MsqEm, generatedEmission, continueEvolution = Generate_Emission(np.sqrt(tEm)*zEm, fac_tEm)

        # if the solver could not find a solution, end the evolution
        if continueEvolution == False:
           return Emissions
           
        # if we have already passed the cutoff this emission does not count
        # this will also terminate the evolution
        if tEm < fac_cutoff*tEm_min: 
           
            zEm = 1.
            pTsqEm = 0.
            QsqEm = 0.
        
            return Emissions

        # if the emission was successful, append to the Emissions list and continue
        if zEm != 1.0:
            
            Emissions.append([math.sqrt(tEm), zEm, math.sqrt(pTsqEm), math.sqrt(MsqEm)])
            Nem = Nem + 1
            
    return Emissions



In [9]:
import ROOT
import math as m


#funzione che mi ritorna gli angoli di emissione, cioè quelli nel sistema in cui il quark è lungo l'asse z'

def angles(pt,z,Ei):

    sin_theta=pt/m.sqrt(pt**2+z**2*Ei**2)
    phi=2*np.pi*random()
    return m.asin(sin_theta),phi

#funzione che ritorna il quadrivettore nella giusta direzione

def rotated_3p(P):

    p_mag=np.sqrt(P[0]**2+P[1]**2+P[2]**2)

    u=P[0]/p_mag
    v=P[1]/p_mag
    w=P[2]/p_mag
    
    theta=P.Theta()
    phi=P.Phi()

    if w!=1:
        u_star=u*m.cos(theta)+(1./m.sqrt(1-w**2))*m.sin(theta)*(u*w*m.cos(phi)-v*m.sin(phi))
        v_star=v*m.cos(theta)+(1./m.sqrt(1-w**2))*m.sin(theta)*(v*w*m.cos(phi)+u*m.sin(phi))
        w_star=w*m.cos(theta)-m.sqrt(1-w**2)*m.sin(theta)*m.cos(theta)

    return p_mag*ROOT.TVector3(u_star,v_star,w_star)


#funzione che aggiorna il 4-impulso del gluone dopo l'emissione dati z e pt generati e il quadrivettore prima dell'emissione e ritorna il 
#quadrimpulso del gluone 


def generate_4vectors(pq_old,pT,z):

    Ei=pq_old[3]
   
    E_q_new= z*Ei
    pz_new=m.sqrt(z**2*Ei**2-pT**2)

    theta_em,phi_em=angles(pT,z,Ei)

    p_q_new=ROOT.TLorentzVector(pT*m.cos(phi_em),pT*m.sin(phi_em),pz_new,E_q_new)
    
    p3_new=rotated_3p(p_q_new)

    pq_old_tmp=ROOT.TLorentzVector(pq_old)

    pq_old.SetPxPyPzE(p3_new[0],p3_new[1],p3_new[2],p_q_new[3])

    pg=pq_old_tmp-pq_old

    return pg



Welcome to JupyROOT 6.26/06


In [10]:
def pt_sign(list):
    
    for i in range(len(list)):
        r=random()
        if r>0.5:
            list[i][2]=(-1)*list[i][2]

In [11]:
 #terrò sempre questo quadrivettore aggiornandolo

def get_gluons(event,pq):

    first_emission=True
    gluons_list=[]

    for emission in event:

        pT=emission[2]
        z=emission[1]
        
        if first_emission==True:
            
            E=pq.E()
            phi0=2*np.pi*random()
            pq.SetPxPyPzE(pT*np.cos(phi0),pT*np.sin(phi0),z*E,z*E)

            pg_first=ROOT.TLorentzVector(-pT*np.cos(phi0),-pT*np.sin(phi0),(1-z)*E,(1-z)*E)
            gluons_list.append(pg_first)
            first_emission==False

        if first_emission==False:

            pg=generate_4vectors(pq,pT,z)
            gluons_list.append(pg)


    return gluons_list


   

            
    

In [12]:
def get_Thrust(gluon_list):

    n=len(gluon_list)


    if(n!=0):

        pt_sum=0
        pmag_sum=0

        for i in range(n):
            
           
            p=gluon_list[i].Vect()
            
            #p.Print()
            n=ROOT.TVector3(0,0,1)
            p_dot_n=np.abs(p.Dot(n))
            #print(p_dot_n)
            pt_sum=pt_sum +p_dot_n
            pmag_sum=pmag_sum+p.Mag()
        
        Thrust=pt_sum/pmag_sum
        return Thrust
    else:
        return 0
  


In [13]:



print('Evolving', Nevolve, 'branches from Q=', Q, 'GeV --> Qc=', Qc, 'GeV')

T_all=[]


for j in tqdm(list(range(Nevolve))): 

    pq=ROOT.TLorentzVector(0,0,Q,Q)

    # quark lungo z>0
    Emissions_zpos =Evolve(Q)
    
    pt_sign(Emissions_zpos)
    
    gluon_list_zpos=get_gluons(Emissions_zpos,pq)

    #quark lungo z<0
    Emissions_zneg=Evolve(Q)
    pt_sign(Emissions_zneg)

    gluon_list_zneg=get_gluons(Emissions_zneg,pq)
     

    gluon_list=gluon_list_zpos+gluon_list_zneg

    T=get_Thrust(gluon_list)
    #print(T)
    T_all.append(T)
    

  



Evolving 10000 branches from Q= 1000.0 GeV --> Qc= 1.0 GeV


  5%|▍         | 470/10000 [00:30<10:18, 15.42it/s]


KeyboardInterrupt: 

In [ ]:
hist_thrust=ROOT.TH1D("Thrust","Thrust",190,0.8,1)

for i in range(len(T_all)):
    hist_thrust.Fill(T_all[i])


canvas = ROOT.TCanvas("canvas", "Canvas", 800, 600)

canvas.Update()
canvas.Draw()
hist_thrust.SetStats(0)
hist_thrust.SetLineWidth(2)
hist_thrust.SetLineColor(ROOT.kGreen)
hist_thrust.Draw()